## 如何使用DNA Center API 部署SDA

### 安装Python环境
Python 3.8/3.9

### 安装包 Python package for DNAC
```
pip install notebook
pip install dnacentersdk
pip install RISE
```


### 检查python 环境和安装包是否 OK
```
pip list
```
DNAC SDK 手册: [https://dnacentersdk.readthedocs.io/en/latest/installation.html](https://dnacentersdk.readthedocs.io/en/latest/installation.html)

DNAC 如何使用API实现SDA功能： [Software Defined Access Guide](https://developer.cisco.com/docs/dna-center/#!sda/software-defined-access-guide)

##### 如何优雅地在notebook中使用Markdown

In [ ]:
import json
from IPython.display import display, Markdown

In [ ]:
def printmd(string):
    display(Markdown(string))

In [ ]:
_test_str = '''
|第一列|第2列|Column 3| 4th|
|:------:|:---------:|------|----------|
|第一行列1|第一行列2|列3|列4|
|第2行列1|第2行列2|列3|列4|
'''
printmd(_test_str)

### DNAC 基本信息设置：IP地址、用户、密码

In [ ]:
dnac =  {
    "base_url": "https://1.1.1.1/",
    "username": "admin",
    "password": "cisco",
    "version": "1.3.3"
}

# 版本选择清单为：1.2.10, 1.3.0, 1.3.1, 1.3.3, 2.1.1 and 2.1.2。

In [ ]:
print(dnac)

# 检查方法很简单，print能够正常输出设置的信息即可。

### DNAC API 交互

#### API 系统初始设置

In [ ]:
import urllib3
from dnacentersdk import DNACenterAPI, ApiError
urllib3.disable_warnings()

def dnac_api(dnac=dnac):
    try:
        dnac_api = DNACenterAPI(
            username=dnac["username"],
            password=dnac["password"],
            base_url=dnac["base_url"],
            version=dnac["version"],
            verify=False
        )
        return dnac_api
    except ApiError as e:
        logger.error(f"Error occurs when touch with DNAC: {e}")
        token["status"] = "error"
        return
api = dnac_api()

# 运行时没有报错即OK。
# 下述步骤中，使用API与DNAC交互时，将使用该名字：api

#### 获取DNAC中交换机等设备详细信息

##### 纳管设备数量

In [ ]:
print(api.devices.get_device_count())

# 输出DNAC中纳管设备总数量，查看输出的 JSON response
# 延伸问题：如何使用SDK获取信息？SDK使用方法？
# 参考SDK手册

##### 设备清单

In [ ]:
devices_dict = {}
devices = api.devices.get_device_list()

_device_list_table = '''
|hostname|IP地址|UUID|
|:-------|:----|:---|
'''

_device_only_ip = ("172.16.0.2","172.16.0.3") # 这2个IP地址是过滤条件
for device in devices.get("response"):
    if device.get("managementIpAddress") in _device_only_ip:
        _new_deivce = "|".join([device.get("hostname"), device.get("managementIpAddress"), device.get("id")]) + "\n"
        _device_list_table += _new_deivce
    if device.get("managementIpAddress"):
        devices_dict[device.get("managementIpAddress")] = device

printmd(_device_list_table)
        
# 输出DNAC中纳管设备具体的hostname名称、IP地址、UUID
# 该信息通过markdown table输出表格信息
# 只列出列表中的设备信息，列表名称为：_device_only_ip

#### 获取DNAC中 site name信息

In [ ]:
print(api.sites.get_site_count())

# 输出DNAC中design site总数量

In [ ]:
sites = api.sites.get_site()
site_siteNameHierarchy = [ i.get("siteNameHierarchy") for i in sites.get("response")]

print("\n".join(site_siteNameHierarchy))

# 输出DNAC中具体的site name
# 删除site name 中的空格

### 设备端口

##### 端口数量

In [ ]:
device_ip = "172.16.0.3"
print(f'该步骤中查询的设备IP地址为: {device_ip}, 其主机名hostname: {devices_dict[device_ip].get("hostname")}')

device_id = devices_dict[device_ip].get("id")
interface_count = api.devices.get_device_interface_count_by_id(device_id)
print('返回的JSON数据为：')
print(interface_count)

# 获取某个设备的端口总数量，本例子中查询的设备IP地址为：172.16.0.3
# 返回设备总数量 response

##### 端口清单

In [ ]:
interfaces = api.devices.get_interface_info_by_id(device_id)
interface_list = []

_interface_list_table = '''
|端口名称|IP地址|Admin status|状态|端口模式|
|:-------|:----|:---|:---|:---|
'''

_interface_only_list = ["GigabitEthernet1/0/6","GigabitEthernet1/0/7","GigabitEthernet1/0/8"]

for i in interfaces.get("response"):
    if "Ethernet Port" in i.get("portType") and "routed" not in i.get("portMode"):
        if i.get("portName","") in _interface_only_list:
            _ip = i.get("ipv4Address","") if i.get("ipv4Address") else ''
            _interface_list_table += "|".join([i.get("portName",""), _ip, i.get("adminStatus",""), i.get("status",""), i.get("portMode","")]) + "\n"
        interface_list.append(i.get("portName"))

printmd(_interface_list_table)

# 获取某个设备的端口详细信息、和运行状态
# 只列出列表中的端口信息，列表名称为：_interface_only_list

### fabric 信息查询

##### control node 检查

In [ ]:
print(f'查询的设备IP地址为: {device_ip}, 其主机名: {devices_dict[device_ip].get("hostname")}\n返回的JSON数据为：')

sda_device = api.sda.get_control_plane_device(device_ip)

print(sda_device)

# 查询该设备是否为：control node，也就是LISP MAP server
# 本例子中，该IP地址设备为Edge node，而不是control node，返回状态为：failed

##### control node 检查

In [ ]:
control_device_ip = "172.16.0.2"
print(f'查询的IP地址为: {control_device_ip}, 其主机名: {devices_dict[control_device_ip].get("hostname")}\n返回的JSON数据为：')

sda_device = api.sda.get_control_plane_device(control_device_ip)

print(sda_device)

# 查询该设备是否为：control node，也就是LISP MAP server。如果正确，则返回success。
# 本例子中，该IP地址设备为control node，返回状态正确，并且返回roles：['BORDERNODE', 'MAPSERVER']

##### edge node 检查

In [ ]:
device_edge_ip = "172.16.0.3"
print(f'查询的IP地址为: {device_edge_ip}, 其主机名: {devices_dict[device_edge_ip].get("hostname")}\n返回的JSON数据为：')

sda_edge_device = api.sda.get_device_info(device_edge_ip)

print(sda_edge_device)

# 查询该设备是否为：edge node。如果正确，则返回success。
# 本例子中，该IP地址设备为edge node，返回状态正确，并且返回roles：['EDGENODE']

##### 默认authentication获取

In [ ]:
siteNameHierarchy = sda_edge_device.get("siteNameHierarchy")
sda_authentication = api.sda.get_default_authentication_profile(siteNameHierarchy)

print(sda_authentication)
print(f'使用点"."来获取默认认证方式为：{sda_authentication.authenticateTemplateName}')

# 查询fabric 所使用的default authentication method
# 本例子中，返回的缺省认证模式为closed，所以：所有没有明确配置的交换机端口将使用802.1x认证方式
# 本例子中，我们也可以使用. 来调用返回的json数据。

### fabric 端口配置查询

In [ ]:
user_access_interface = "GigabitEthernet1/0/9"

print(f"查询的SDA配置端口为：{user_access_interface}\n其返回数据为：")
_data_ok = api.sda.get_port_assignment_for_access_point(device_edge_ip, user_access_interface)
print(json.dumps(_data_ok, indent=4))

# 查询SDA设置的端口配置信息：status == success表示端口已经配置，
# dataIpAddressPoolName: GC_wired，为用户端口配置ip pool name
# pool name 是 dnac 中已经设置OK的

In [ ]:
no_config_interface = "GigabitEthernet1/0/8"

print(f"查询的SDA配置端口为：{no_config_interface}\n其返回数据为：")
_data = api.sda.get_port_assignment_for_access_point(device_edge_ip, no_config_interface)
print(json.dumps(_data, indent=4))

# 查询SDA设置的端口配置信息：status == success表示端口已经配置，failed表示端口未明确配置，采用默认的认证方式。


### SDA 端口设置操作

##### 配置准备

In [ ]:
print(f'拟打算使用API配置的空闲端口为:{no_config_interface}\n该端口将使用的配置参数为：')
default_authentication = sda_authentication.get("authenticateTemplateName")
no_authentication = "No Authentication"


new_port_assignment_info = {k:v for k,v in _data_ok.items() if k in ["siteNameHierarchy","deviceManagementIpAddress","dataIpAddressPoolName","voiceIpAddressPoolName"]}
new_port_assignment_info.update({"interfaceName": no_config_interface, "authenticateTemplateName": no_authentication})

print(json.dumps(new_port_assignment_info, indent=4))

# 准备配置端口，准备端口的配置信息，格式为JSON数据

##### 端口配置

In [ ]:
port_assignment = [new_port_assignment_info]
sda_new_int_info = api.sda.add_port_assignment_for_user_device(payload=port_assignment)
print(json.dumps(sda_new_int_info, indent=4))

# 端口设置：使用API设置端口，端口设置为：接入的pool name为 GC_wired
# 返回"status": "pending"表示：DNAC 正在后台处理请求，并返回查询的链接URL
# 查询处理结果，浏览器打开executionStatusUrl

##### API结果查询

In [ ]:
_data = api.custom_caller.call_api('GET',sda_new_int_info.executionStatusUrl).response
print(json.dumps(_data, indent=4))

# 查询刚刚API 配置的结果
# isError == false 表示运行处理正确，没有问题
# startTime、endTime 分别表示提交申请、处理完毕时间戳

In [ ]:
print(f'刚刚配置的端口为：{no_config_interface}\n配置信息是：')

fabric_name = sda_site.get("fabricName")
sda_assign_int_info = api.sda.get_port_assignment_for_access_point(device_edge_ip, no_config_interface)
print(json.dumps(sda_assign_int_info, indent=4))

# 查询：该端口目前的状态，下述显示为端口配置：SDA成功配置

##### 端口配置删除

In [ ]:
print(no_config_interface)

fabric_name = sda_site.get("fabricName")
sda_delete_int_info = api.sda.delete_port_assignment_for_user_device(device_edge_ip, no_config_interface)
print(json.dumps(sda_delete_int_info, indent=4))

# 删除：端口的配置，下述显示为端口配置删除任务进行中。可以查询任务结果executionStatusUrl，和前述步骤类似。

##### 遍历查询所有端口方法

In [ ]:
print("interface          \tSDA_config_status\n-----------------------------------------")
i = 1
for one in interface_list:
    sda_assign_int_info = api.sda.get_port_assignment_for_access_point(device_edge_ip, one)
    print(f'{one}:\t{sda_assign_int_info.status}')
    i += 1
    if i >5:
        break

# rate limit for this API。注意点：限速
# 查询所有端口SDA配置